In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pylab as plt

# Not sure I'll need all this, grabbing it all just in case...

import astropy.io.fits as fits
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.coordinates import get_sun, get_body, get_moon

import pytz

from astroplan import Observer, FixedTarget, time_grid_from_range, observability_table, moon_illumination
from astroplan import AirmassConstraint, MoonSeparationConstraint

In [2]:
apo = Observer.at_site('apo')

In [3]:
reference_time = Time('2018-5-19')

In [4]:
tz_1 = pytz.timezone("America/Denver")

In [5]:
constraints = [MoonSeparationConstraint(25*u.deg)]
constraints.append(AirmassConstraint(2))

In [6]:
astro_set1 = apo.twilight_evening_astronomical(reference_time, which='nearest')
astro_rise1 = apo.twilight_morning_astronomical(reference_time, which='next')
observing_length1 = (astro_rise1 - astro_set1).to(u.h)

print("Astronomical Evening Twilight starts at {0.iso} UTC".format(astro_set1))
print("Astronomical Morning Twilight starts at {0.iso} UTC".format(astro_rise1))
print("You can observe for {0:.1f} during the observing window.".format(observing_length1))

Astronomical Evening Twilight starts at 2018-05-19 03:33:27.086 UTC
Astronomical Morning Twilight starts at 2018-05-19 10:25:44.668 UTC
You can observe for 6.9 h during the observing window.


In [7]:
print(apo.moon_phase(reference_time))

2.30130362773722 rad


In [8]:
# right on

In [9]:
apo.moon_illumination(reference_time)

0.16637621157343807

In [10]:
# much better than last time we tried this

In [11]:
obs_range = [astro_set1, astro_rise1]

# Here is where the target list of galaxies may be found:

In [17]:
target_array = [['M49', 12.5, 8.0],
               ['M89',  12+35/60+39.8/3600, 12+33/60+23/3600],
               ['NGC 4214', 12+(15/60)+(39.2/3600), 69+(27/60)+(45/3600)],
               ['NGC 4449', 12+(28/60)+(11.118/3600), 44+(5/60)+(36.81/3600)]]

In [18]:
target_array

[['M49', 12.5, 8.0],
 ['M89', 12.59438888888889, 12.55638888888889],
 ['NGC 4214', 12.26088888888889, 69.4625],
 ['NGC 4449', 12.469755, 44.093558333333334]]

In [13]:
targets = [FixedTarget(coord=SkyCoord(ra = RA*u.hr, dec = DEC*u.deg), name=Name)
           for Name, RA, DEC in target_array]

In [14]:
observing_table = observability_table(constraints, apo, targets, time_range=obs_range)

print(observing_table)

target name ever observable always observable fraction of time observable
----------- --------------- ----------------- ---------------------------
        M49            True             False          0.6428571428571429
        M89            True             False          0.6428571428571429
   NGC 4214            True             False          0.9285714285714286
   NGC 4449            True             False          0.7857142857142857


In [15]:
for i, target_table in enumerate(targets):

    if observing_table['ever observable'][i]:
        name = observing_table['target name'][i]
        obj_frac = observing_table['fraction of time observable'][i]
        obj_time = obj_frac * observing_length1
        output = "You can observe {0:s} for {1:.2f}".format(name, obj_time.to(u.h))
        print(output)

You can observe M49 for 4.42 h
You can observe M89 for 4.42 h
You can observe NGC 4214 for 6.38 h
You can observe NGC 4449 for 5.40 h


In [16]:
# M101 was a spur-of-the-moment thing, so of course I didn't plan for it in here. And M100 was Jocelin's idea,
# ergo it is also absent.